# Download

https://mapreader.readthedocs.io/en/latest/User-guide/Download.html

In [ ]:
from mapreader import SheetDownloader

my_ts = SheetDownloader(
    metadata_path="../NLS_metadata/metadata_OS_One_Inch_GB_WFS_light.json",
    download_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png",
)

In [ ]:
my_ts.extract_wfs_id_nos()
my_ts.plot_all_metadata_on_map(map_extent="uk", add_id=False)

In [ ]:
my_ts.get_minmax_latlon()

In [ ]:
my_ts.extract_published_dates()
my_ts.metadata["published_date"].hist(bins=30, edgecolor="k")


## Query maps using a list of lats/lons

In [5]:
my_ts.query_map_sheets_by_coordinates((-4.33, 55.90))
my_ts.query_map_sheets_by_coordinates((-3.25, 51.93), append=True)

In [ ]:
my_ts.print_found_queries()

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

## Download map tiles

In [ ]:
my_ts.get_grid_bb(zoom_level=14)
my_ts.download_map_sheets_by_queries(force=True)

# Load

https://mapreader.readthedocs.io/en/latest/User-guide/Load.html

## Load in downloaded maps

In [9]:
from mapreader import loader

In [ ]:
my_files = loader("./maps/*png")

In [ ]:
# len() shows the total number of images currently read (or sliced, see below)
print(f"Number of images: {len(my_files)}")

In [ ]:
print(my_files)

In [ ]:
my_files.add_metadata("./maps/metadata.csv")

In [ ]:
my_files.check_georeferencing()
my_files.georeferenced

In [15]:
parent_list = my_files.list_parents()

## Patchify maps

In [ ]:
my_files.patchify_all(patch_size=100)  # in pixels

In [ ]:
my_files.show_sample(num_samples=6, tree_level="patch")

In [ ]:
my_files.show_patches(
    parent_id=parent_list[0],
    figsize=(15, 15)
)

In [ ]:
my_files.explore_patches(
    parent_id=parent_list[0],
    xyz_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png"
)

## Calculate pixel intensities

In [ ]:
my_files.calc_pixel_stats()

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True)

In [ ]:
parent_df.head()

In [ ]:
patch_df.head()

In [ ]:
my_files.show_patches(
    parent_list[0],
    column_to_plot="mean_pixel_R",
    vmin=0.75,
    vmax=0.9,
    figsize=(20, 20),
    alpha=0.5,
)

In [ ]:
my_files.explore_patches(
    parent_list[0],
    column_to_plot="mean_pixel_R",
    xyz_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png",
    vmin=0.75,
    vmax=0.9,
)

# Annotate
https://mapreader.readthedocs.io/en/latest/User-guide/Annotate.html

## Set up inputs

In [26]:
from mapreader import Annotator

In [ ]:
annotator = Annotator(
    patch_paths="./patches_100_pixel/*png",
    parent_paths="./maps/*png",
    metadata_path="./maps/metadata.csv",
    annotations_dir="./annotations_one_inch",
    labels=["no", "railspace"],
    username="rw",
    task_name="railspace",
)

In [ ]:
annotator.annotate(show_context=True)

# Train
https://mapreader.readthedocs.io/en/latest/User-guide/Classify.html

## Read annotations

In [29]:
from mapreader import AnnotationsLoader

In [ ]:
annotated_images = AnnotationsLoader()

annotated_images.load("./annotations_one_inch/rail_space_#rw#.csv", images_dir="./patches_100_pixel")

In [ ]:
annotated_images.annotations

In [ ]:
print(annotated_images)

In [ ]:
# show sample images for target label (tar_label)
annotated_images.show_sample(label_to_show="railspace")

## Prepare datasets and dataloaders

In [ ]:
annotated_images.create_datasets(
    frac_train=0.7, 
    frac_val=0.2, 
    frac_test=0.1, 
    context_datasets=True, 
    context_df="./patch_df.csv"
)

In [ ]:
annotated_images.datasets["train"].patch_df.head()

In [ ]:
annotated_images.labels_map

In [ ]:
dataloaders = annotated_images.create_dataloaders(batch_size=8)

## Set up `ClassifierContainer`

### Load a pretrained PyTorch model, dataloaders and labels map

In [38]:
from mapreader import ClassifierContainer

In [ ]:
my_classifier = ClassifierContainer(model ="resnet18",
                                    labels_map={0: 'No', 1: 'railspace'},
                                    dataloaders=dataloaders,
                                    )

In [ ]:
my_classifier.labels_map

In [ ]:
my_classifier.model_summary()

### Set up loss function, optimizer and scheduler

In [ ]:
my_classifier.add_loss_fn("cross-entropy")

In [44]:
params_to_optimize = my_classifier.generate_layerwise_lrs(
    min_lr=1e-4, max_lr=1e-3, spacing="geomspace"
)

In [45]:
my_classifier.initialize_optimizer(params2optimize=params_to_optimize)

In [46]:
my_classifier.initialize_scheduler()

## Train/fine-tune model

**Note:** it is possible to interrupt a training (using Kernel/Interrupt in Jupyter Notebook or ctrl+C). 

In [ ]:
my_classifier.train(
    num_epochs=20,
    save_model_dir="./models_tutorial",
    tensorboard_path="tboard_tutorial",
    tmp_file_save_freq=5,
    remove_after_load=False,
    print_info_batch_freq=10,
)

In [ ]:
my_classifier.list_metrics()

In [ ]:
my_classifier.plot_metric(
    metrics=["loss"],
)

In [ ]:
my_classifier.plot_metric(
    metrics=['precision_macro', 'recall_macro','fscore_macro','support_macro','rocauc_macro'],
)

# Model inference

Here, we show how to load an already trained/fine-tuned model and a dataset and then do model inference.

## Load patches and add metadata

In [52]:
from mapreader import load_patches

In [ ]:
my_maps = load_patches(
    "./patches_100_pixel/*74488689*png", parent_paths="./maps/map_74488689.png"
)

my_maps.add_metadata("./maps/metadata.csv", ignore_mismatch=True)
my_maps.add_metadata("patch_df.csv", tree_level="patch", ignore_mismatch=True)

In [54]:
parent_df, patch_df = my_maps.convert_images()

## Create context dataset from patches

In [55]:
from mapreader import PatchContextDataset

In [56]:
patch_dataset = PatchContextDataset(patch_df, total_df=patch_df, transform="val", create_context=True)


## Inference

In [57]:
my_classifier.load_dataset(patch_dataset, set_name="patches")

In [ ]:
my_classifier.inference(set_name="patches")

In [ ]:
my_classifier.show_inference_sample_results(
    label="railspace", num_samples=9, set_name="patches", min_conf=80
)

### Save predictions

In [ ]:
my_classifier.save_predictions("patches")

## Add outputs to `MapImages` as metadata

In [ ]:
my_maps.add_metadata("./patches_predictions_patch_df.csv", tree_level="patch")

In [ ]:
my_maps.add_shape()

In [ ]:
parent_df, patch_df = my_maps.convert_images()
parent_df.head()

In [ ]:
parent_list = my_maps.list_parents()

my_maps.show_patches(
    parent_list[0],
    column_to_plot="predicted_label",
    categorical=True,
    figsize=(15, 15),
    alpha=0.5,
)

In [ ]:
my_maps.explore_patches(
    parent_list[0],
	column_to_plot="predicted_label",
	categorical=True,
	xyz_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png",
)

In [ ]:
my_maps.show_patches(
    parent_list[0],
    column_to_plot="conf",
    figsize=(15, 15),
    alpha=0.5,
)

In [ ]:
my_maps.explore_patches(
    parent_list[0],
	column_to_plot="conf",
	xyz_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png",
)